In [1]:
import pandas as pd 
import requests
import os 
from dotenv import load_dotenv, find_dotenv
import pickle
load_dotenv(find_dotenv())
import sys
sys.path.append('/Users/kangsukwoo/fconline/utils/')
import useful_function

headers = {'x-nxopen-api-key' : os.getenv('x-nxopen-api-key')}

# data = pd.DataFrame()
data = list()
for num in range(1,6):
    if num == 1:
        num = ''
    with open(f'/Users/kangsukwoo/fconline/python_crawling/picklefile/matches_detail{num}.pickle', 'rb') as file:
        matches_detail_data = pickle.load(file)
    data.extend(matches_detail_data)    

In [2]:
matches = [] 
for match in data:
    for individual_match in match['matchInfo'] :
        matches.append({'matchId' : match['matchId'],
                          'matchDate' : match['matchDate'], 
                          **individual_match['matchDetail'],
                          **individual_match['shoot'],
                          'shootDetail' : individual_match['shootDetail'],
                          **individual_match['pass'],
                          **individual_match['defence'],
                          'player' : individual_match['player']
                        })

In [3]:
player_status_list = []
for match in matches:
    for player in match['player']:
        player_status_list.append({
            'matchId' : match['matchId'],
            'matchDate' : match['matchDate'],
            'matchResult' : match['matchResult'],
            'spId' : player['spId'],
            'spPosition' : player['spPosition'],
            'spGrade' : player['spGrade'],
            **player['status']
            })

In [80]:
player_data = pd.DataFrame(player_status_list)

# date & patch parsing
player_data = player_data[player_data['matchDate'] >= '2024-02-15'] 
player_data['patch_boolean'] = player_data.apply(lambda x : 'before_patch' if x['matchDate'] < '2024-02-22 06:00:00' else 'after_patch', axis=1)

# parse seasonid
player_data['seasonId'] = player_data['spId'].apply(lambda x : int(str(x)[:3]))

# use metadata
spid = useful_function.get_metadata('spid')
spposition = useful_function.get_metadata('spposition') 
seasonid = useful_function.get_metadata('seasonid') 

# merge dataframe
player_data = pd.merge(spid, player_data, left_on='id', right_on='spId').drop(columns='id') 
player_data = pd.merge(spposition, player_data, left_on='spposition', right_on='spPosition').drop(columns='spposition')
player_data = pd.merge(seasonid, player_data, on='seasonId')

# select atk position data
player_data = player_data[player_data['spPosition'].isin(range(17,28))]

In [81]:
player_data = player_data[['seasonId','className','desc','name','matchId','matchDate','matchResult','spId','spPosition','spGrade','shoot','effectiveShoot','assist','goal','dribble','spRating','patch_boolean']]

In [85]:
striker_position = [24, 25, 26] # st, rs, ls
midfielder_position = [16, 19, 27, 12, 17, 23] # lm, lw, lam, rm rw, ram 

In [96]:
player_data['position_category'] = player_data['spPosition'].apply(lambda x : 'st' if x in striker_position else ('mid' if x in midfielder_position else None))

before_patch = player_data[player_data['patch_boolean'] == 'before_patch']
after_patch = player_data[player_data['patch_boolean'] == 'after_patch']

before_patch_silver = before_patch[before_patch['spGrade'].isin([5,6,7])]
before_patch_silver = before_patch_silver.groupby(['spId','spPosition']).filter(lambda x : x['matchId'].count() >= 100)
before_patch_gold = before_patch[before_patch['spGrade'] >= 8]
before_patch_gold = before_patch_gold.groupby(['spId','spPosition']).filter(lambda x : x['matchId'].count() >= 100)

after_patch_silver = after_patch[after_patch['spGrade'].isin([5,6,7])]
after_patch_silver = after_patch_silver.groupby(['spId','spPosition']).filter(lambda x : x['matchId'].count() >= 100)
after_patch_gold = after_patch[after_patch['spGrade'] >= 8]
after_patch_gold = after_patch_gold.groupby(['spId','spPosition']).filter(lambda x : x['matchId'].count() >= 100)

In [98]:
grouped_before_patch_silver = before_patch_silver.groupby(['seasonId','className','name','position_category'], as_index=False).agg({
    'shoot' : 'mean', 
    'effectiveShoot' : 'mean',
    'assist' : 'mean',
    'goal' : 'mean', 
    'dribble' : 'mean', 
    'spRating' : 'mean',
    'matchId' : 'count'
})

grouped_before_patch_gold = before_patch_gold.groupby(['seasonId','className','name','position_category'], as_index=False).agg({
    'shoot' : 'mean', 
    'effectiveShoot' : 'mean',
    'assist' : 'mean',
    'goal' : 'mean', 
    'dribble' : 'mean', 
    'spRating' : 'mean',
    'matchId' : 'count'
})

grouped_after_patch_silver = after_patch_silver.groupby(['seasonId','className','name','position_category'], as_index=False).agg({
    'shoot' : 'mean', 
    'effectiveShoot' : 'mean',
    'assist' : 'mean',
    'goal' : 'mean', 
    'dribble' : 'mean', 
    'spRating' : 'mean',
    'matchId' : 'count'
})

grouped_after_patch_gold = after_patch_gold.groupby(['seasonId','className','name','position_category'], as_index=False).agg({
    'shoot' : 'mean', 
    'effectiveShoot' : 'mean',
    'assist' : 'mean',
    'goal' : 'mean', 
    'dribble' : 'mean', 
    'spRating' : 'mean',
    'matchId' : 'count'
})

In [101]:
grouped_before_patch_silver.to_excel('grouped_before_patch_silver.xlsx')
grouped_before_patch_gold.to_excel('grouped_before_patch_gold.xlsx')
grouped_after_patch_silver.to_excel('grouped_after_patch_silver.xlsx')
grouped_after_patch_gold.to_excel('grouped_after_patch_gold.xlsx')